In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import pandas as pd
import folium


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
df=pd.read_html(url, header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Drop cells with Borough that are "Not assigned":

In [4]:
df=df[df.Borough!='Not assigned']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [5]:
df.reset_index(drop=True, inplace=True)

In [6]:
df.head(5)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# Make "Not assigned" value in Neighbourhood the same as Borough

In [7]:
df['Neighbourhood']=df['Neighbourhood'].replace(['Not assigned'],"Queen's Park")
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [8]:
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [9]:
df=df.groupby(['Postcode','Borough'],as_index=False,).agg(lambda x:",".join(x))

In [10]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df.shape

(103, 3)

# Get the latitude and the longitude coordinates of each neighborhood

Generate a dataframe from the csv file

In [28]:

d = pd.read_html('https://github.com/vtn2113/Week-3-part-1/blob/master/Geospatial_Coordinates.csv')

df.coor = d[0]

df.coor.head()

,Unnamed: 0,Postal Code,Latitude,Longitude
0,NaN,M1B,43.806686,-79.194353
1,NaN,M1C,43.784535,-79.160497
2,NaN,M1E,43.763573,-79.188711
3,NaN,M1G,43.770992,-79.216917
4,NaN,M1H,43.773136,-79.239476


Change column name "Postal Code" to "Postcode"

In [46]:
df.coor.rename(columns={'Postal Code':'Postcode'}, inplace=True)

In [47]:
df.coor.head()

,Unnamed: 0,Postcode,Latitude,Longitude
0,NaN,M1B,43.806686,-79.194353
1,NaN,M1C,43.784535,-79.160497
2,NaN,M1E,43.763573,-79.188711
3,NaN,M1G,43.770992,-79.216917
4,NaN,M1H,43.773136,-79.239476


In [51]:
df_coor = df.coor.drop(df.coor.columns[0], axis=1)

In [56]:
#Merge the Wikipedia table with the coordinates table
df_new = df.merge(df_coor, on="Postcode", how="left")
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
